In [ ]:
%%sh
pip -q install pandas scikit-learn joblib

In [ ]:
import sagemaker

print(sagemaker.__version__)

sess   = sagemaker.Session()
bucket = sess.default_bucket()                     
prefix = 'sklearn-boston-housing-dep'

In [ ]:
training_path = sess.upload_data(path='housing.csv', key_prefix=prefix + "/training")
output_path   = 's3://{}/{}/output/'.format(bucket,prefix)

print(training_path)
print(output_path)

# Adding source files for training

In [ ]:
import sagemaker
from sagemaker.sklearn import SKLearn

sk = SKLearn(entry_point='sklearn-boston-housing.py',
             framework_version='0.23-1',
             source_dir='src',
             role=sagemaker.get_execution_role(),
             instance_count=1, 
             instance_type='ml.m5.large',
             output_path=output_path,
             hyperparameters={
                  'normalize': True,
                  'test-size': 0.1
              }
)

sk.fit({'training':training_path})

# Adding libraries for training

In [ ]:
%%sh
mkdir -p lib
pip install -q -t lib joblib

In [ ]:
import sagemaker
from sagemaker.sklearn import SKLearn

sk = SKLearn(entry_point='sklearn-boston-housing-joblib.py',
             framework_version='0.23-1',
             source_dir='src',
             dependencies=['lib/joblib'],
             role=sagemaker.get_execution_role(),
             instance_count=1, 
             instance_type='ml.m5.large',
             output_path=output_path,
             hyperparameters={
                  'normalize': True,
                  'test-size': 0.1
              }
)

sk.fit({'training':training_path})

In [ ]:
sk_predictor = sk.deploy(initial_instance_count=1, 
                         instance_type='ml.t2.medium')

In [ ]:
import pandas as pd

data = pd.read_csv('housing.csv')
payload = data[:10].drop(['medv'], axis=1) 
payload = payload.to_csv(header=False, index=False)

sk_predictor.serializer = sagemaker.serializers.CSVSerializer()
sk_predictor.deserializer = sagemaker.deserializers.CSVDeserializer()

response = sk_predictor.predict(payload)
print(response)

In [ ]:
sk_predictor.delete_endpoint()